In [3]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog,ttk
import os
import time
import Levenshtein as LV
from glob import glob
import re
import csv
##=================================== Define Functions ===========================================
def convertN(sequence): # 由于简并碱基的复杂程度太高且不常见，将所有简并碱基视为N
    return re.sub(r'[^ATCG]', 'N', sequence)
#===============================================================================
def translate(ncltseq,continu = 'N'):  # dna序列翻译到蛋白质序列, continu = 'N'则遇到终止密码子后不继续翻译，否则继续
    codon_table = {'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M','ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K','AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L','CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q','CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'*', 'TAG':'Q', 'TGC':'C', 'TGT':'C', 'TGA':'*', 'TGG':'W',
        'GCN':'A', 'CGN':'R', 'GGN':'G', 'CTN':'L', 'CCN':'P', 'TCN':'S', 'GTN':'V'}
    bplength = len(ncltseq)
    output = ''
    i = 0
    while i in range (0,bplength):
        codon = ncltseq[i:i+3]
        if codon in codon_table.keys():
            aminoacid = codon_table[codon]
        else:
            aminoacid = 'X'
        output = output + aminoacid
        i = i+3
    return output

def findcds(seq,consb,consa,cv_er):
    try:
        fidx = seq.index(consb)+len(consb)
    except:
        val0 = cv_er
        fidx = False
        for i in range(len(seq)-len(consb)):
            char = seq[i:i+len(consb)]
            val = LV.distance(char,consb)
            if val <= val0:
                fidx = i
                val0 = val
    try:
        ridx = seq.index(consa)
    except:
        val0 = cv_er
        ridx = False
        for i in range(len(seq)-len(consa)):
            char = seq[i:i+len(consa)]
            val = LV.distance(char,consa)
            if val <= val0:
                ridx = i
                val0 = val
    if fidx != False and ridx != False:
        cds = seq[fidx:ridx]
        cds = convertN(cds)
        return cds
    else:
        return 'NA'

def deduplicate(info,atpl,al_er,total):
    atpl = min(atpl,total)
    total = min(total,len(info))
    out = pd.DataFrame()   # <out> as the deduplicated collection
    out.loc[0,'library seq']=info['library seq'][0]
    out.loc[0,'count'] = info['count'][0]
    outidx = 1
    for i in range(1,total): # reduce calculation steps if too many sequence types
        seq = info['library seq'][i]
        if seq == 'NA':
            continue
        scount = info['count'][i]
        status = False
        for j in range(0,min(atpl,i,len(out))):
            err = LV.distance(out['library seq'][j],seq)
            if err <= al_er:
                out.loc[j,'count'] += scount
                status = True
                break
        if status == False: # if template not found in <out>
            out.loc[outidx,'library seq'] = seq
            out.loc[outidx,'count'] = scount
            outidx += 1
    out = out.sort_values(by = 'count', ascending = False)
    return out

def dedupprot(info,atpl,al_er,total):
    atpl = min(atpl,total)
    total = min(total,len(info))
    out = pd.DataFrame()   # <out> as the deduplicated collection
    out.loc[0,'protein seq']=info['protein seq'][0][0]
    out.loc[0,'count'] = info['count'][0]
    outidx = 1
    for i in range(1,total): # reduce calculation steps if too many sequence types
        seq = info['protein seq'][i][0]
        if seq == 'NA' or seq == '*':
            continue
        scount = info['count'][i]
        seq_lib = list(out['protein seq'])
        if seq in seq_lib:
            idx = seq_lib.index(seq)
            out.loc[idx,'count'] += scount
        else:
            out.loc[outidx,'protein seq'] = seq
            out.loc[outidx,'count'] = scount
            outidx += 1
    out = out.sort_values(by = 'count', ascending = False)
    return out
#===============================================================================
##=================================== GUI Functions ===========================================
def preview_txt():
    folder_path = filedialog.askdirectory()
    try:
        sstr = os.path.join(folder_path, "**",'*√.txt')
    except:
        sstr = os.path.join(folder_path, "**",'*.txt')
    global file_list
    file_list = glob(sstr, recursive=True)
    first_five_lines = ''
    count = 0
    with open(file_list[0], 'r',buffering = 1) as f1:
        for line in f1:
            first_five_lines += str(count+1)+ '. '+line
            count += 1
            if count == 10:
                break
    file_path_entry.delete(0, tk.END)
    file_path_entry.insert(0, str(file_list))
    text_box.delete('1.0', tk.END)
    text_box.insert(tk.END, first_five_lines)
    
def show_default_consv():    ## Set default values
    cb_entry.delete(0, tk.END)
    cb_entry.insert(0,'CATGGCG')
    ca_entry.delete(0, tk.END)
    ca_entry.insert(0,'GGTTCTG')

def use_default_param():
    cv_er_entry.delete(0, tk.END)
    cv_er_entry.insert(0,'2')
    ab_tpl_entry.delete(0, tk.END)
    ab_tpl_entry.insert(0,'50')
    align_er_entry.delete(0, tk.END)
    align_er_entry.insert(0,'4')
    prcss_num_entry.delete(0, tk.END)
    prcss_num_entry.insert(0,'20000')
    
def use_default_param2():
    scale.set(66)
    cluster_cnt_entry.delete(0, tk.END)
    cluster_cnt_entry.insert(0,'200')
#============================ Processing ==========================================    
def submit1():  # translation
    global file_list
    consb = cb_entry.get().upper()
    consa = ca_entry.get().upper()
    cv_er = int(cv_er_entry.get())
    atpl = int(ab_tpl_entry.get())
    al_er = int(align_er_entry.get())
    total = int(prcss_num_entry.get()) 
    for file in file_list:
        print('>> Processing file: '+file)
        try:
            readin = pd.read_csv(file,header = None)
        except:
            readin = pd.read_csv(file,header = None,sep = '  ',engine = 'python')
        statistics = readin[0].value_counts()
        statistics.to_csv(file.replace('√.txt','-nt_stat.csv'),header = False)
        print('>> Finished abundance stat of DNA sequences, start extracting library sequences and process deduplication.')
        stidx = statistics.index.tolist()
        stcnt = list(statistics)
        info = pd.DataFrame()
        info['DNA seq'] = stidx
        info['count'] = stcnt
        stcds = []
        baddata = []
        for i in range(len(stidx)):
            seq = stidx[i]
            count = stcnt[i]
            cds = findcds(seq,consb,consa,cv_er)
            if cds == 'NA':
                baddata.append([count,'conserved seq not found',seq,'n.a.','n.a.'])
            stcds.append(cds)
        info['library seq'] = stcds
        info = deduplicate(info,atpl,al_er,total)
        info.to_csv(file.replace('√.txt','-nt_stat(dedup).csv'),header = True, index = False)
        print('>> Finished deduplication of library DNA sequences, start translation stat.')
        stprot = []
        for i in range(len(info)):
            cds = info['library seq'][i]  # the <info> collection has been deduplicated
            count = info['count'][i]
            if len(cds)%3 != 0:
                baddata.append([count,'frame shift','full seq deduplicated',cds,'n.a.'])
                stprot.append(['NA'])
                continue
            prot = translate(cds,continu = 'Y')
            if '*' in prot:
                baddata.append([count,'in-frame stop codon','full seq deduplicated',cds,prot])
                stprot.append(['*',prot])
                continue
            stprot.append([prot])
        info['protein seq'] = stprot
        info = dedupprot(info,atpl,al_er,total)
        info.to_csv(file.replace('√.txt','-aa_stat.csv'),header = True, index = False)
        print('>> Finished translation stat.')
        csvfile = open(file.replace('√.txt','-baddata.csv'),'w',newline='')
        write = csv.writer(csvfile)
        write.writerow(['Count','Problem','Full DNA Seq','Library DNA Seq','Library Protein Seq'])
        write.writerows(baddata)
        csvfile.close()
        statistics, info, baddata = [],[],[]
    react.config(text = 'Translation Finished.')
##############################################################################
def submit2():
    global file_list
    simi = scale.get()/100
    total = int(cluster_cnt_entry.get())
    i = 0
    for file in file_list:
        groups = {}
        data = []
        try:
            file_path = file.replace('√.txt','-aa_stat.csv') # new version
        except:
            file_path = file.replace('.txt','-aa_stat.csv') # old version
        with open(file_path,'r') as f:
            for row in f:
                data.append(row.strip(' \n').split(','))
                i += 1
                if i == total:
                    break
        if data[0][0] == 'protein seq':
            del data[0]
        groups[data[0][0]] = [] # representative seqs stored as keys of a dict
        for row in data:
            seq = row[0]
            if seq == '':
                continue
            state = False
            for rep_seq in groups.keys():
                dif = LV.distance(seq,rep_seq)/max(len(seq),len(rep_seq))
                if dif <= 1-simi:   # if the difference between two pepseqs is smaller than preset value
                    similarity = '{:.0f}%'.format(100*(1-dif))
                    cys_cnt = seq.count('C')
                    row = row+[similarity,cys_cnt]
                    groups[rep_seq].append(row)
                    state = True
                    break
                continue
            if state == False:  # if not assigned, add a new group
                similarity = '100%'
                cys_cnt = seq.count('C')
                row = row+[similarity,cys_cnt]
                groups[seq] = [row]
        csvfile = open(file_path.replace('-aa_stat.csv','-aa_clusters.csv'),'w',newline='')
        write = csv.writer(csvfile)
        write.writerow(['Peptide Seq','Quantity','Similarity','Cysteine Count'])
        for i in range(len(list(groups.keys()))):
            key = list(groups.keys())[i]
            cnt = 0
            for j in groups[key]:
                cnt += int(float(j[1]))
            write.writerow(['Family '+str(i+1)+': '+key+', Total Quantity: '+str(cnt)])
            write.writerows(groups[key])
    csvfile.close()
    print('>> Finished clustering stat.')
    react2.config(text = 'Clustering Finished.')
#===============================================================================


root = tk.Tk()
root.title('NGS peptide - S4 Translation and Clustering')
# Create entry field and button for input data file selection dialogue
file_path_entry = tk.Entry(root, width = 85)
file_path_entry.grid(row=0, column=3, columnspan=85, padx=10, pady=5)
select_file_button = tk.Button(root, text="    Select File and Preview  ", command=preview_txt)
select_file_button.grid(row=0, column= 86, columnspan = 35, padx=10, pady=5)

# Create a text box to display the first 5 lines of the text file
text_box = tk.Text(root, height=6, width=120)
text_box.grid(row=3, column=0, columnspan=120, padx=10, pady=5)

hyfen = tk.Label(root,text = '========== Translation Section ============================================================================= ',font = ("Arial", 10, "bold"))
hyfen.grid(row=4,column = 0, columnspan = 125, padx = 10, pady = 5)

# Create a text box to display default seq format
tpl_box = tk.Text(root, height=4, width=80)
tpl_box.insert(tk.END,'   Barcode        N-ter conserved              Library       C-ter conserved\n  ======== ---------------------------- ==================== ---------------\n  AGGAGTCC ATTCTATGCGGCCCAGCCGGCCATGGCG NNKTGC (NNK)n TGTNNK GGTTCTGGCGCTGAA\n            |F||Y||A||A||Q||P||A||M||A| |X||C|  |X|n  |C||X| |G||S||G||A||E|')
tpl_box.grid(row = 5, column = 30, columnspan = 80)
tpl_label = tk.Label(root,text = 'The default format\n of a sequence:')
tpl_label.grid(row = 5, column =0, columnspan = 30, rowspan = 3)

############### Set/Show default template format
hyfen2 = tk.Label(root,text = '* ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ')
hyfen2.grid(row=8,column = 0, columnspan = 120, padx = 10, pady = 5)

cb_label = tk.Label(root, text='Conserved Seq before Library:' )
cb_label.grid(row=9, column=0, columnspan=30, padx=5, pady=5)
cb_entry = tk.Entry(root)
cb_entry.grid(row=10, column=0, columnspan=30, padx=5, pady=5)
clup_label = tk.Label(root, text='Library:' )
clup_label.grid(row=9, column=30, columnspan=30, padx=5, pady=5)
cl_label = tk.Label(root, text='NNKTGC (NNK)n TGTNNK' )
cl_label.grid(row=10, column=30, columnspan=30, padx=5, pady=5)
ca_label = tk.Label(root, text='Conserved Seq after Library:' )
ca_label.grid(row=9, column=60, columnspan=30, padx=5, pady=5)
ca_entry = tk.Entry(root)
ca_entry.grid(row=10, column=60, columnspan=30, padx=5, pady=5)
df_button = tk.Button(root,text = ' Use Default  \n  Conserved Seq ',command = show_default_consv)
df_button.grid(row = 9, column = 90, columnspan= 30, rowspan = 2, padx = 5, pady=5)

hyfen3 = tk.Label(root,text = '* ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ----------- * ')
hyfen3.grid(row=11,column = 0, columnspan = 120, padx = 10, pady = 5)

cv_error = tk.Label(root, text = 'Max Error per Conserved Seq (bp):')
cv_error.grid(row = 12, column = 0, columnspan= 35, padx = 5, pady=5)
cv_er_entry = tk.Entry(root, width = 5)
cv_er_entry.grid(row = 12, column = 35, columnspan= 10, padx = 5, pady=5)
ab_tpl = tk.Label(root, text = 'Abundant Template Count:')
ab_tpl.grid(row = 12, column = 45, columnspan= 30, padx = 5, pady=5)
ab_tpl_entry = tk.Entry(root, width = 5)
ab_tpl_entry.grid(row = 12, column = 75, columnspan= 10, padx = 5, pady=5)
align_error = tk.Label(root, text = 'Max Deduplication Error (bp):')
align_error.grid(row = 13, column = 0, columnspan= 35, padx = 5, pady=5)
align_er_entry = tk.Entry(root, width = 5)
align_er_entry.grid(row = 13, column = 35, columnspan= 10,padx = 5, pady=5)
prcss_num = tk.Label(root,text = 'Process Count:')
prcss_num.grid(row = 13, column = 45, columnspan= 25, padx = 5, pady=5)
prcss_num_entry = tk.Entry(root,width = 10)
prcss_num_entry.grid(row = 13, column = 70, columnspan= 20, padx = 5, pady=5)
dfp_button = tk.Button(root,text='    Use Default   \n    Parameters   ',command = use_default_param)
dfp_button.grid(row = 12, column = 93, columnspan = 25, rowspan = 2, padx = 5, pady=5 )

submit_button = tk.Button(root,text='        >>> Submit for Translation <<<       ',command = submit1)
submit_button.grid(row = 14, column = 5, columnspan = 65, padx = 5, pady=10 )
react = tk.Label(root,text = '')
react.grid(row = 14, column = 70, columnspan = 20, padx = 5, pady=10 )

hyfen = tk.Label(root,text = '========== Clustering Section ============================================================================== ',font = ("Arial", 10, "bold"))
hyfen.grid(row=15,column = 0, columnspan = 125, padx = 10, pady = 5)

scale = tk.Scale(root, from_=0, to=100, orient=tk.HORIZONTAL,showvalue = True, length=300, width=10)
scale.grid(row = 16, column = 25, columnspan= 30, padx = 5, pady=5)
stringency = tk.Label(root, text = 'Grouping Stringency\n (Similarity, %):  \n\n')
stringency.grid(row = 16, column = 0, columnspan= 25,rowspan=2, padx = 5, pady=5)

cluster_cnt = tk.Label(root, text = 'Process Count:')
cluster_cnt.grid(row = 16, column = 55, columnspan= 20, padx = 5, pady=5)
cluster_cnt_entry = tk.Entry(root, width = 5)
cluster_cnt_entry.grid(row = 16, column = 75, columnspan= 10, padx = 5, pady=5)
dfpc_button = tk.Button(root,text='  Use Default Parameters  ',command = use_default_param2)
dfpc_button.grid(row = 16, column = 88, columnspan = 28, padx = 5, pady=5 )

submit2_button = tk.Button(root,text='        >>> Submit for Clustering <<<       ',command = submit2)
submit2_button.grid(row = 17, column = 5, columnspan = 65, padx = 5, pady=10 )
react2 = tk.Label(root,text = '')
react2.grid(row = 17, column = 70, columnspan = 20, padx = 5, pady=10 )

root.mainloop()



>> Finished clustering stat.
